In [1]:
# Load packages
import time
from scipy.stats import uniform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer, accuracy_score

from lightgbm import LGBMClassifier
import lightgbm as lgbm

from hyperopt import hp, fmin, tpe
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold
from bayes_opt import BayesianOptimization

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
# !conda install -c conda-forge imbalanced-learn -y

In [3]:
# Make scorer: accuracy
acc_score = make_scorer(accuracy_score)

In [4]:
# Load dataset
trainSet = pd.read_csv('train.csv')
testSet = pd.read_csv('test.csv')
submitSet = pd.read_csv('sample_submission.csv')

trainSet.head()

# Remove not used variables
train = trainSet.drop(columns=['Name', 'Ticket'])
train['Cabin_letter'] = train['Cabin'].str[0:1]
train['Cabin_no'] = train['Cabin'].str[1:]

train.head()

# Feature generation: training data
train = trainSet.drop(columns=['Name', 'Ticket', 'Cabin'])
train = train.dropna(axis=0)
train = pd.get_dummies(train)

train.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
2,2,0,3,0.33,1,2,71.29,0,1,0,0,1
3,3,0,3,19.00,0,0,13.04,0,1,0,0,1
4,4,1,3,25.00,0,0,7.76,0,1,0,0,1
5,5,0,2,35.00,0,0,6.71,0,1,0,0,1
6,6,0,3,7.00,0,0,9.77,0,1,0,0,1


In [5]:
train.groupby('Survived').size()

Survived
0    55093
1    41239
dtype: int64

In [6]:
# train validation split
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=['PassengerId','Survived'], axis=0),
                                                  train['Survived'],
                                                  test_size=0.2, random_state=111,
                                                  stratify=train['Survived'])

In [7]:
sm = SMOTE(random_state = 42)
lgbm = LGBMClassifier()

In [8]:
# pipeline = Pipeline([('transformer', sm), ('estimator', lgbm)])

In [9]:
# lgbm_pip = make_pipeline(sm, lgbm)
# lgbm_pip = make_pipeline(SMOTE(random_state = 42),LGBMClassifier())
lgbm_pip = make_pipeline(SMOTE(random_state = 42), GradientBoostingClassifier())


In [10]:
# myModel = make_pipeline(SMOTE(random_state = 42),LGBMClassifier())

# skfold = StratifiedKFold (n_splits = 3)
# accs = cross_val_score (myModel, X_train, y_train, cv = skfold)

In [11]:
params_gbm = {}
learning_rate = (0.01, 1)
params_gbm['learning_rate'] = learning_rate

In [12]:
lgbm_pip

Pipeline(steps=[('smote', SMOTE(random_state=42)),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [14]:
cross_val_score(lgbm_pip,
                             X_train, y_train, scoring='roc_auc', cv=5)

array([0.84020428, 0.84453879, 0.84311761, 0.83738358, 0.83866223])

In [26]:
# Gradient Boosting Machine
# def gbm_cl_bo(max_depth, max_features, learning_rate, n_estimators, subsample):
def gbm_cl_bo(learning_rate):
    params_gbm = {}
    
#     params_gbm['max_depth'] = round(max_depth)
#     params_gbm['max_features'] = max_features
    params_gbm['learning_rate'] = learning_rate
#     params_gbm['n_estimators'] = round(n_estimators)
#     params_gbm['subsample'] = subsample
#     lgbm_pip.fit(X_train, y_train)

    model_pip = make_pipeline(SMOTE(random_state = 42), GradientBoostingClassifier(random_state=42, **params_gbm))
    
    scores = cross_val_score(LGBMClassifier(random_state=123, **params_gbm),
                             X_train, y_train, scoring=acc_score, cv=5).mean()
    
    score = scores.mean()
    return score

In [27]:
# Run Bayesian Optimization
start = time.time()

params_gbm ={
#     'max_depth':(3, 10),
#     'max_features':(0.8, 1),
    'learning_rate':(0.01, 1)#,
#     'n_estimators':(80, 150)#,
#     'subsample': (0.8, 1)
}

gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm, random_state=111)
# gbm_bo.maximize(init_points=20, n_iter=4)

# print('It takes %s minutes' % ((time.time() - start)/60))

In [28]:
gbm_bo

In [29]:
gbm_bo.maximize(init_points=20, n_iter=4)

print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | learni... |
-------------------------------------
|  1        |  0.7602   |  0.616    |
|  2        |  0.7704   |  0.1774   |
|  3        |  0.7663   |  0.4417   |
|  4        |  0.7571   |  0.7716   |
|  5        |  0.768    |  0.3024   |
|  6        |  0.7705   |  0.1577   |
|  7        |  0.7711   |  0.03225  |
|  8        |  0.7659   |  0.426    |
|  9        |  0.7706   |  0.2463   |
|  10       |  0.7684   |  0.3443   |
|  11       |  0.7511   |  0.9908   |
|  12       |  0.7696   |  0.2453   |
|  13       |  0.7726   |  0.09038  |
|  14       |  0.759    |  0.6729   |
|  15       |  0.7594   |  0.625    |
|  16       |  0.7698   |  0.2815   |
|  17       |  0.7652   |  0.4716   |
|  18       |  0.7719   |  0.1272   |
|  19       |  0.772    |  0.08322  |
|  20       |  0.7528   |  0.9018   |
|  21       |  0.7698   |  0.2626   |
|  22       |  0.7716   |  0.1096   |
|  23       |  0.7721   |  0.05508  |
|  24       |  0.7701   |  0.01     |
It takes 0.4

In [21]:
params_gbm = gbm_bo.max['params']

# params_gbm['max_depth'] = round(params_gbm['max_depth'])
# params_gbm['n_estimators'] = round(params_gbm['n_estimators'])

params_gbm

{'learning_rate': 0.08321798850177216,
 'max_depth': 9.305419254124786,
 'n_estimators': 135.577379233574}

In [40]:
# params_gbm = gbm_bo.max['params']

# # params_gbm['max_depth'] = round(params_gbm['max_depth'])
# # params_gbm['n_estimators'] = round(params_gbm['n_estimators'])

# params_gbm

In [15]:
from bayes_opt import BayesianOptimization
# Gradient Boosting Machine
def gbm_cl_bo(max_depth, max_features, learning_rate, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['max_features'] = max_features
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    
    model_pip = make_pipeline(SMOTE(random_state = 42), GradientBoostingClassifier(random_state=42, **params_gbm))
    
    scores = cross_val_score(model_pip,
                             X_train, y_train, scoring='roc_auc', cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_gbm ={
    'max_depth':(3, 10),
    'max_features':(0.8, 1),
    'learning_rate':(0.01, 1),
    'n_estimators':(80, 150),
    'subsample': (0.8, 1)
}
gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm, random_state=111)
gbm_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | learni... | max_depth | max_fe... | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.8314   |  0.616    |  4.183    |  0.8872   |  133.8    |  0.8591   |
|  2        |  0.8411   |  0.1577   |  3.157    |  0.884    |  96.71    |  0.8675   |
|  3        |  0.8095   |  0.9908   |  4.664    |  0.8162   |  126.9    |  0.9242   |
|  4        |  0.8372   |  0.2815   |  6.264    |  0.8237   |  85.18    |  0.9802   |
|  5        |  0.7634   |  0.796    |  8.884    |  0.963    |  149.4    |  0.9155   |
|  6        |  0.8021   |  0.8156   |  5.949    |  0.8055   |  111.8    |  0.8211   |
|  7        |  0.7834   |  0.819    |  7.884    |  0.9131   |  99.2     |  0.9997   |
|  8        |  0.8381   |  0.1467   |  7.308    |  0.897    |  108.4    |  0.9456   |
|  9        |  0.8312   |  0.3296   |  5.804    |  0.8638   |  146.3    |  0.9837   |
|  10       |  0.8335   |  0.8157   |  3.239    |  0.9

In [16]:
gbm_bo.max['params']

{'learning_rate': 0.07864837617488214,
 'max_depth': 5.5531193523155205,
 'max_features': 0.8723008386644597,
 'n_estimators': 113.04444496691903,
 'subsample': 0.8358969695415375}

In [20]:
import sklearn

In [21]:
sklearn.__version__

'0.22.2.post1'

In [42]:
from bayes_opt import BayesianOptimization
# Gradient Boosting Machine
def gbm_cl_bo(max_depth, max_features, learning_rate, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['max_features'] = max_features
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(GradientBoostingClassifier(random_state=123, **params_gbm),
                             X_train, y_train, scoring=acc_score, cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_gbm ={
    'max_depth':(3, 10),
    'max_features':(0.8, 1),
    'learning_rate':(0.01, 1),
    'n_estimators':(80, 150),
    'subsample': (0.8, 1)
}
gbm_bo = BayesianOptimization(gbm_cl_bo, params_gbm, random_state=111)
gbm_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | learni... | max_depth | max_fe... | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.7647   |  0.616    |  4.183    |  0.8872   |  133.8    |  0.8591   |
|  2        |  0.7711   |  0.1577   |  3.157    |  0.884    |  96.71    |  0.8675   |


KeyboardInterrupt: 